### **1) 여러 예외 묶어서 처리 + 로그 남기기**

사용자로부터 두 개의 값을 입력받아 나눗셈을 수행하는 프로그램을 작성하시오.

- `ValueError`(정수 변환 실패), `ZeroDivisionError`(0으로 나누기)를 하나의 except 블록에서 처리
- 예외 메시지를 `error.log` 파일에 한 줄씩 기록
- 정상 계산되면 결과 출력

**Hint**

- `except (ValueError, ZeroDivisionError) as e:` 형태 사용

- 파일 기록은 `with open("error.log", "a")`

- 문자열 → 숫자 변환은 `int()` 사용

---

In [15]:
from datetime import datetime
def divide_with_logging():
  try:
    a=int(input("첫번째 숫자 : "))
    b=int(input("두번째 숫자 : "))
    result = a/b
    print(a/b)
  except (ZeroDivisionError, ValueError) as e:
    print(e)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("error.log","a",encoding="utf-8") as log:
      log.write(f"{timestamp} - {type(e).__name__} : {e}\n")

divide_with_logging()


division by zero


### **2) 사용자 정의 예외 + 다단계 검사**

`register_user(age, email)` 함수를 다음 조건에 맞게 작성하시오.

- 나이가 0 미만이면 `NegativeValueError` 발생
- 나이가 120 초과면 `AgeLimitError` 발생
- 이메일에 `"@"` 없으면 `EmailFormatError` 발생
- 위 예외들은 모두 `UserValidationError`를 상속
- try 블록에서 호출하고, 각 자식 예외마다 다른 메시지를 출력

**Hint**

- 사용자 정의 예외: `class MyError(Exception): pass`

- 이메일 검사: `"@" not in email`

- 예외 계층 구조 설계 필요 (부모 → 자식)

In [23]:
class UserValidationError(Exception):
  pass

class NegativeValueError(UserValidationError):
  pass
class AgeLimitError(UserValidationError):
  pass
class EmailFormatError(UserValidationError):
  pass

def register_user(age, email):
  if age < 0 :
    raise NegativeValueError("나이는 0보다 커야 합니다.")
  if age > 120: 
    raise AgeLimitError("나이는 120보다 작아야 합니다.")
  if "@" not in email :
    raise EmailFormatError("이메일 형식에 맞지 않습니다.")
  print(f"유저등록완료 : age={age},email={email}")

register_user(-10,"fdsfk@djfjdksjf.com")


NegativeValueError: 나이는 0보다 커야 합니다.

### **3) 파일 여러 개 열기 + 도중 실패 시 리소스 정리**

두 개의 파일을 번갈아 읽어 출력하는 프로그램 작성.

- 첫 번째 파일은 정상적으로 열리지만 두 번째 파일이 없을 수 있음
- 두 번째 파일에서 오류가 나도 **이미 열린 첫 번째 파일은 반드시 닫아야 함**
- `with` 사용 금지 → `try-except-finally` 구조만 사용

**Hint**

- 첫 번째 파일을 변수에 저장 후 두 번째 파일 open 시도

- finally 블록에서 `if f1:` 으로 파일 닫기

- 예외 발생 여부와 상관없이 정리 코드가 실행되도록 설계

In [ ]:
def read_file():
  f1 = None
  f2 = None
  try:
    file01 = input("첫번째 파일 이름 : ")
    file02 = input("두번째 파일 이름 : ")
    f1 = open(file01,"r",encoding="utf-8")
    f2 = open(file01,"r",encoding="utf-8")
  except FileNotFoundError as e:
    print("파일을 열 수 없습니다",e)
  finally:
    if f1 is not None and not f1.closed:
      f1.close()
    if f2 is not None and not f2.closed:
      f2.close()

read_file()

파일을 열 수 없습니다 [Errno 2] No such file or directory: 'fdsfds'


### **4) try-return vs finally-return 비교**

`calc()` 함수를 작성하시오.

- try 블록에서 값을 `return`
- finally 블록에서도 다른 값을 `return`
- 실제 반환되는 값을 출력하고 이유를 설명

**Hint**

- finally의 return은 try의 return을 덮어쓴다

- 코드 흐름을 print로 확인하면 이해 쉬움

In [ ]:
def calc():
  try:
    print("try 블록 실행")
    return "try 블록에서 리턴"
  finally:
    print("finally 블록 실행")
    return "finally 블록에서 리턴"
result = calc();
#만약 finally에 리턴값이 있으면 try에 있는 리턴값은 무시된다.
print(result)  


try 블록 실행
finally 블록 실행
try 블록에서 리턴


### **5) 파일 확장자 검사 + 사용자 정의 예외**

사용자로부터 파일명을 입력받아 검사하시오.

- 파일 이름은 반드시 `.txt` 로 끝나야 함
- 그렇지 않으면 `InvalidExtensionError` 예외 발생
- 확장자가 맞으면 파일 열어서 내용 출력
- 파일이 없으면 `FileNotFoundError` 처리
- 오류가 나도 종료되지 않고 반복 입력

**Hint**

- 확장자 검사: `filename.endswith(".txt")`

- 사용자 정의 예외: `raise InvalidExtensionError("...")`

- 무한 루프: `while True:` + try/except

In [29]:
class InvalidExtensionError(Exception):
  pass
def read_file_extension():
  while True:
    file_name = input("읽을 파일 이름을 쓰시오 exit를 쓰면 종료합니다.").strip()
    if file_name.lower() == "exit":
      print("종료합니다.")
      break
    try:
      if not file_name.endswith(".txt"):
        raise InvalidExtensionError("txt 파일만 읽을 수 있습니다.")
      
      with open(file_name,"r",encoding="utf-8") as f:
        content = f.read()
        print(content)
    except InvalidExtensionError as e:
      print(e)
    except FileNotFoundError as e:
      print("파일을 찾을 수 없습니다.")
    except Exception as e:
      print("알 수 없는 오류 : ",e)
read_file_extension()

txt 파일만 읽을 수 있습니다.
txt 파일만 읽을 수 있습니다.
파일을 찾을 수 없습니다.
종료합니다.


### **6) 입력 검증 + 예외 재발생(re-raise)**

정수 리스트 `nums = [10, -3, 5, -1, 9]` 를 처리하는 프로그램 작성.

1. `process_number(n)` 함수 작성
2. 음수 → `NegativeValueError` 발생 후 메시지 출력 및 re-raise
3. 100 초과 → `TooLargeError` 발생
4. 리스트 전부를 처리하고 예외가 발생해도 다음 값으로 넘어가기
5. 마지막에는 `"처리 완료"` 출력

**Hint**

- 예외 재발생은 `raise` 단독 사용

- for 반복문 내부에 try/except

- except 안에서 continue 활용 가능

In [32]:
class NegativeValueError(Exception) :
  pass

class TooLargeError(Exception) :
  pass

def process_number(n):
  if n<0 :
    print("음수 발견")
    raise NegativeValueError(f"음수 : {n}")
  if n > 100 :
    print("100보다 큰 수 발견")
    raise NegativeValueError(f"큰수 : {n}")
  print("정상처리 : ",n)

def run_process_numbers():
  nums = [10, -3, 5, -1, 9,102]
  for num in nums:
    try:
      process_number(num)
    except NegativeValueError as e:
      print("NegativeValueError : ",e)
    except TooLargeError as e:
      print("TooLargeError : ",e)
    except Exception as e:
      print("알 수 없는 오류 : ", e)
  print("정상 처리 완료")
run_process_numbers()

정상처리 :  10
음수 발견
NegativeValueError :  음수 : -3
정상처리 :  5
음수 발견
NegativeValueError :  음수 : -1
정상처리 :  9
100보다 큰 수 발견
NegativeValueError :  큰수 : 102
정상 처리 완료


### **7) API 재시도(retry) 로직 구현**

가끔 `ConnectionError`가 발생하는 `call_api()` 함수가 있다고 가정.

- 최대 3번까지 재시도
- 실패할 때마다 `"재시도 중..."` 출력
- 3번 실패하면 예외 재발생
- 기타 예외는 재시도 없이 즉시 중단

**Hint**

- while 또는 for로 재시도 횟수 관리

- 예외 재발생: `raise`

- 랜덤하게 에러 테스트하려면 `random.choice([True, False])`

In [ ]:
import random
import time

# random.choice([True,False])
def call_api():
  if random.choice([True,True]):
    raise ConnectionError("네트워크 오류 발생 !!!")
  return "응답데이터 잘 받음"
def run_call_api(max_retries=3, delay=3):
  for attempt in range(1,max_retries+1):
    try:
      print(f"{attempt}번째 api call 시도...")
      response = call_api()
      print("api 호출 성공 : ",response)
      return response
    except ConnectionError as e:
      print("ConnectionError 발생 : ",e)
      if(attempt < max_retries):
        print(f"{delay}초 후 재시도 중...")
        time.sleep(delay)
      else:
        print("api 호출 실패 - 재시도 횟수를 초과했습니다.")
        raise
    except Exception as e:
      print("알 수 없는 오류")
      raise

run_call_api()

1번째 api call 시도...
ConnectionError 발생 :  네트워크 오류 발생 !!!
3초 후 재시도 중...
2번째 api call 시도...
ConnectionError 발생 :  네트워크 오류 발생 !!!
3초 후 재시도 중...
3번째 api call 시도...
ConnectionError 발생 :  네트워크 오류 발생 !!!
api 호출 실패 - 재시도 횟수를 초과했습니다.


ConnectionError: 네트워크 오류 발생 !!!